In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)


In [8]:
#Khai bao thu vien
import cv2
import numpy as np
from ultralytics import YOLO
from tracker import *
import pandas as pd
import os

In [16]:
model = YOLO('yolov8n.pt') #Khai bao model
tracker = Tracker() #Ham giup tinh toan va tra ra gia tri trung tam anh bounding box
count = 0

100%|██████████| 6.23M/6.23M [00:00<00:00, 114MB/s]


In [36]:
cap = cv2.VideoCapture(r"/highway.mp4")

In [13]:
down = {}
up = {}
counter_down = []
counter_up = []
red_line_y = 200 # toa do duong ke do
blue_line_y = 270 #toa do duong ke xanh
offset = 6
if not os.path.exists('detected_frames'): #duong dan luu anh
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

In [42]:
import time


In [43]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
from google.colab.patches import cv2_imshow
while True:
    ret,frame = cap.read() #Doc frame
    if not ret:
        break
    count +=1
    frame = cv2.resize(frame, (1020, 500)) #Resize lai anh
    results = model.predict(frame) #Cho frame vao model de xu ly
    a = results[0].boxes.data #Chuyen doi ket qua tra ve cua model thanh dang bang
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []
    print('hi')
    for index , row in px.iterrows():
       #Toa do xac dinh vi tri bounding box :
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d] #Name cua phuong tien
        if 'car' in c : #Neu co xe oto:
            list.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list) #Update cac toa do can thiet cho viec tinh toan van toc cua xe oto
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox #toa do cua tung xe
        cx =int (x3 + x4) //2 #diem trung tam bounding box
        cy = int(y3+y4) // 2  #diem trung tam bounding box

        ##Xe di xuong :
        if red_line_y < (cy+offset) and red_line_y > (cy-offset): #Tinh toan hoang cach cua xe voi duong mau do
            down[id] = time.time()
        if id in down:
          if blue_line_y < (cy+offset) and blue_line_y > (cy-offset):  #Tinh toan hoang cach cua xe voi duong mau xanh
            elapsed_time= time.time() - down[id]  # dung ham time de tinh thoi gian
            if counter_down.count(id) == 0:  #Neu xe chua xuat hien
              counter_down.append(id)
              distance = 10  #Khoang cach
              a_speed_ms = distance / elapsed_time # v = s/t (m/s)
              a_speed_kh = a_speed_ms * 3.6 #(km/h)
              cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #Ve center point
              cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2) #Ve bounding box
              cv2.putText(frame,str(id),(x3,y3),(x4,y4),(0,255,0),2) #Nhap id xe
              cv2.putText(frame,str(int(a_speed_kh) + 'Km/h'),(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2) #Nhap van toc xe
        ##Xe di len:
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:
           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
              counter_up.append(id)
              elapsed1_time=time.time() - up[id]
              a_speed_ms1 = distance1 / elapsed1_time
              a_speed_kh1 = a_speed_ms1 * 3.6
              cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
              cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
              cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
              cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    text_color = (0, 0, 0)  # Mau text
    yellow_color = (0, 255, 255)  # Mau background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines
    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)
    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == 27:
    #if cv2.waitKey(0) & 0xFF == 27:
        break
cap.release()
out.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.